# InvokeAI in Google Colab
Make sure to enable GPU, Edit > Notebook Settings > Hardware accelerator > GPU <br>
This takes about 10-12 Mins to start fully.<br>
To start, Click "Runtime" > "Run All".<br>
Adapted from:
https://github.com/jenkinsmichpa/invokeai-colab/blob/main/invokeai-colab.ipynb <br>



In [ ]:
#@title 1. Configuration { display-mode: "form" }
#@markdown <u>Instance Type</u><br>
#@markdown Persistent = Stores images in your Google drive, in a folder called "/invokeaiOutput".<br>
#@markdown Temp = Images are stored in the runtime and are removed when the runtime ends or crashes. Make sure to download your images! <br>
Type = "Persistent" #@param ['Persistent','Temp'] {type:"string"}
#@markdown <u>Xformers</u><br>
#@markdown Xformers makes this "memory efficient" so larger images (above 1024x1024) can be made, However it can be less stable and may generate blank outputs. It is recomended that you set it to "Enabled" unless you experiance issues with your models.
xformers = "Enabled" #@param ['Enabled' ,'Disabled'] {type:"string"}
#@markdown <u>NSFW Checker</u><br>
#@markdown This checks outputs for "Not Safe For Work" content and obscures the image.
nsfw_checker = "Enabled" #@param ['Enabled' ,'Disabled'] {type:"string"}
#@markdown <u>Custom checkpoint / safetensor Model</u>
custom_model_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/crosskemonoFurryModel_crosskemono25.safetensors"] {allow-input: true}
custom_vae_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/Crosskemono2.vae.pt"] {allow-input: true}
#@markdown Custom models are stored in /content/invokeai/CustomModels and must be manually added via the GUI after it starts. <br>
#@markdown The Custom VAE will be stored in /content/invokeai/CustomVae <br>
#@markdown <br>
#@markdown <u>NGROK - Recomended, but not required.</u><br>
#@markdown An NGROK token is needed for the more stable connection, however this project gives 2 options on how to connect, if you are happy with the less stable "Localtunnel" Leave this as "none".<br>
#@markdown Sign up and get your free token at https://ngrok.com/
ngrok_token = "None" #@param ['None'] {allow-input: true}

In [ ]:
#@title 2. Build and configure.
#@markdown This takes about 10-12 mins to finish + any extra time to download custom models. { display-mode: "form" }
%%capture
file_path = '/content/invokeai'

#Set up temporary storage if needed
if Type == "Temp":
    outpath = '/content/invokeai/output'
    import os
    if not os.path.exists(file_path):
      os.makedirs(file_path)

# Mount and set up Google drive if needed.
if Type == "Persistent":
  outpath = '/content/drive/MyDrive/invokeaiOutput'
  import os
  from google.colab import drive
  if not os.path.exists(file_path):
    drive.mount('/content/drive')
    if not os.path.exists(file_path):
      os.makedirs(file_path)


#Install Python + Pip

!sudo apt-get update -y
!sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev python3.9-venv -y

#Change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#Update pip
#%cd {file_path}
#!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
#!python3 get-pip.py --force-reinstall/
#!python -m pip install --upgrade pip
#!pip install --upgrade setuptools

#Handle colab dependencies
!ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

#Create InvokeAI root
import os
os.environ['INVOKEAI_ROOT'] = file_path
if not os.path.exists(file_path):
  os.makedirs(invokeai_root)

#Create and prepare virtual environment
%cd {file_path}
!python -m venv .venv --prompt InvokeAI
!source .venv/bin/activate; python -m pip install --upgrade pip
!source .venv/bin/activate; pip install InvokeAI[xformers] --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117
!mkdir {outpath}

#Edit invokeai.init and download default models
!cd {file_path}
!source .venv/bin/activate; invokeai-configure --yes
!sed -i 's/--max_loaded_models=2/--max_loaded_models=1/' invokeai.init
if xformers == "Disabled":
  !sed -i 's/--xformers/--no-xformers/' invokeai.init
if nsfw_checker == "Disabled":
  !sed -i 's/--nsfw_checker/--no-nsfw_checker/' invokeai.init


#Download custom models
if custom_model_url != "None":
  !mkdir {file_path}/CustomModels
  %cd {file_path}/CustomModels
  !wget -nc {file_path}/CustomModels {custom_model_url}

#Download custom VAE
if custom_vae_url != "None":
  !mkdir {file_path}/CustomVae
  %cd {file_path}/CustomVae
  !wget -nc {file_path}/CustomVae {custom_vae_url}

In [ ]:
#@title 3. Connect via NGROK. { display-mode: "form" }
#@markdown This link will start working when the next step tells you to connect to localhost.<br>
#@markdown If this fails, See the next step for an alternate connection method.<br>
#@markdown You can ignore this step if you have not suplied a NGROK token in the configuration.

if ngrok_token != "None":
  %cd {file_path}
  !pip install pyngrok --quiet
  from pyngrok import ngrok
  ngrok.kill()
  ngrok.set_auth_token(ngrok_token)
  public_url = ngrok.connect(9090).public_url
  print(f'InvokeAI Public URL: {public_url}')

if ngrok_token == "None":
  print("You have chosen not to use NGROK.")
  print("Please follow the connection process in step 4")

In [ ]:
#@title 4. Conect via "Localtunnel" + Start InvokeAI { display-mode: "form" }
#@markdown A "localtunnel" Interface connection is generated here, To use this, please do the following: <br>
#@markdown 1) Copy the IP address stated in line 3 of this cells output. <br>
#@markdown 2) Click the random "https://XXX-YYY-ZZZ.loca.lt" link in line 9. <br>
#@markdown 3) Paste the IP into the provided box and submit. <br> <br>
#@markdown Your downloaded model is stored at: /content/invokeai/CustomModels<br>
#@markdown Your downloaded VAE is stored at: /content/invokeai/CustomVae <br>
#@markdown <br>Diffuser models, can be added via the interface, <br>
#@markdown Model manager > Add New > Add Diffusers > Give it any name/Discription > Put your "Hugging face Repo ID" into the "Repo ID" box > Add Model<br>
#@markdown It will auto download and configure when it is first loaded<br><br>
#@markdown <u>An Example Diffuser Model:</u><br>
#@markdown kebab111/crosskemono - A Furry/Anime model<br>
%cd {file_path}
!echo IP address:
!curl ipv4.icanhazip.com
!npm install -g localtunnel
!npx localtunnel --port 9090 & !source .venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath}

In [ ]:
#@title 5. Temp Mode - Zip Output for Download.
#@markdown Google Colab does not give a nice way to download multiple files at once.<br>
#@markdown When running in Temp Mode, This will compress all images into a single "output.zip" to make downloading everything simple!<br>
#@markdown This file can be downloaded using the Colab file browser: /invokeai/output/Output.zip <br>
#@markdown For this step to run you must manually stop step 4.

if Type == "Temp":
  !mkdir /content/zip
  !rm {outpath}/Output.zip
  !rm -r /content/zip/*
  !cp -p {outpath}/*png /content/zip
  !cd /content/zip/
  !zip {outpath}/Output.zip /content/zip/*.png